In [6]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score, mean_squared_error

In [2]:
num_submission = 1 
# Load data
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [7]:
#features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
features = train_df.columns.drop(['ID_code', 'target']).tolist()

In [8]:
train_features_df = train_df[features]
target_series = train_df['target']

In [9]:
train_values_df, holdout_test_df, train_target_series, holdout_test_target_series = train_test_split(
    train_features_df,
    target_series,
    test_size=0.2,
    random_state=0
)

In [4]:
params = {
    'task': 'train', 'max_depth': -1, 'boosting_type': 'gbdt',
    'objective': 'binary', 'num_leaves': 3, 'learning_rate': 0.01,
}

In [13]:
y_hat = 0.0
predictions = {}
scores = {}
for feature in features: # loop over all features
    lgb_train = lgb.Dataset(train_values_df[feature].values.reshape(-1,1), train_target_series)
    gbm = lgb.train(params, lgb_train, 110, verbose_eval=5)
    prediction = gbm.predict(holdout_test_df[feature].values.reshape(-1,1), num_iteration=gbm.best_iteration)
    scores[feature] = roc_auc_score(holdout_test_target_series, prediction)
    predictions[feature] = prediction
    y_hat += prediction

In [ ]:
#y_hat /= len(features)

In [10]:
#sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
#sub["target"] = y_hat
#sub.to_csv('submission{}.csv'.format(num_submission), index=False)

In [12]:
#auc?

In [15]:
scores_features= {score: feature for feature, score in scores.items()}

In [16]:
score_values = scores.values()

In [17]:
score_values

dict_values([0.5416000162937749, 0.5359879372067158, 0.5516876428249474, 0.5015102548737951, 0.5015708996171379, 0.5246125167814942, 0.5636000219151827, 0.5025022221867769, 0.5187715487872698, 0.5386222624748245, 0.49919053113469275, 0.5201803933274056, 0.5607572649564129, 0.5436760949304567, 0.5005892647561467, 0.5197826614241917, 0.508340178712582, 0.5052786956871977, 0.53111869738082, 0.5168183969068855, 0.5184804955567203, 0.5495067097039426, 0.5485330119660385, 0.5234201864363407, 0.5333483401381443, 0.5023154730687356, 0.5503479236001401, 0.5002309069371522, 0.5188978573879877, 0.49753465964771615, 0.49952051884430176, 0.5228981120405265, 0.5194974061357139, 0.5396996412628052, 0.5454472052756165, 0.5304077969921924, 0.5419818877966466, 0.498946681806253, 0.5068256247080087, 0.5048468718846012, 0.5442805866401402, 0.5034919810895845, 0.5087008349396576, 0.521920506316593, 0.5399086129409071, 0.522198776382921, 0.5003470111598235, 0.5089428843099615, 0.53371077209315, 0.5241317652

In [18]:
max_score = max(score_values)

In [19]:
max_score

0.566440646615853

In [20]:
min_score = min(score_values)

In [21]:
min_score

0.4955334904226028

In [22]:
scores_more_05 = {feature: score for feature, score in scores.items() if score > 0.5}

In [23]:
len(scores_more_05)

191

In [24]:
scores_more_055 = {feature: score for feature, score in scores.items() if score > 0.55}

In [25]:
len(scores_more_055)

15

In [26]:
print(scores_more_055)

{'var_2': 0.5516876428249474, 'var_6': 0.5636000219151827, 'var_12': 0.5607572649564129, 'var_26': 0.5503479236001401, 'var_53': 0.5508025064887798, 'var_76': 0.5574129981235298, 'var_81': 0.5631469655295002, 'var_99': 0.5505050980214279, 'var_110': 0.5507570956218899, 'var_139': 0.566440646615853, 'var_146': 0.5523729076559725, 'var_148': 0.5545082118382361, 'var_149': 0.550615944289468, 'var_169': 0.5507175969251841, 'var_174': 0.5511306596256735}
